In [2]:
#prepare - bring in required modules
import pandas as pd
from pathlib import Path

In [3]:
#define csv filepaths
schools_csv = Path("Resources/schools_complete.csv")
students_csv = Path("Resources/students_complete.csv")

#read csv's
schools_data = pd.read_csv(schools_csv)
students_data = pd.read_csv(students_csv)

In [4]:
#view headers to find common column for merging
schools_data.columns

Index(['School ID', 'school_name', 'type', 'size', 'budget'], dtype='object')

In [5]:
#view headers to find common column for merging
students_data.columns

Index(['Student ID', 'student_name', 'gender', 'year', 'school_name',
       'reading_score', 'maths_score'],
      dtype='object')

In [6]:
#merge csv's and view
schools_merged_df = pd.merge(schools_data, students_data, on=["school_name"], how="left")
schools_merged_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,year,reading_score,maths_score
0,0,Huang High School,Government,2917,1910635,0,Paul Bradley,M,9,96,94
1,0,Huang High School,Government,2917,1910635,1,Victor Smith,M,12,90,43
2,0,Huang High School,Government,2917,1910635,2,Kevin Rodriguez,M,12,41,76
3,0,Huang High School,Government,2917,1910635,3,Richard Scott,M,12,89,86
4,0,Huang High School,Government,2917,1910635,4,Bonnie Ray,F,9,87,69


In [7]:
#run calculations on merged data

#total schools
total_schools = schools_merged_df["school_name"].nunique()
total_schools

#total students
total_students = schools_merged_df["Student ID"].nunique()
total_students

#total budget - this is repeated on each line, so can't rely on the sum
first_schools = schools_merged_df.groupby("school_name").first()
total_budget = first_schools["budget"].sum()
total_budget

#average maths score
av_maths_score = schools_merged_df["maths_score"].mean()
av_maths_score

#average reading score
av_read_score = schools_merged_df["reading_score"].mean()
av_read_score

#percentage passing maths - 50% or higer
maths_pass = schools_merged_df.loc[schools_merged_df["maths_score"] >= 50]
maths_percent_pass = (len(maths_pass)/total_students)*100
maths_percent_pass

#percentage passing reading - 50% or higher
read_pass = schools_merged_df.loc[schools_merged_df["reading_score"] >= 50]
read_percent_pass = (len(read_pass)/total_students)*100
read_percent_pass

#percentage passing both maths and reading
both_pass = schools_merged_df.loc[(schools_merged_df["maths_score"] >= 50) & (schools_merged_df["reading_score"] >= 50)]
both_percent_pass = (len(both_pass)/total_students)*100
both_percent_pass

72.80827163645647

In [8]:
#show calulcations in a dataframe

#collate results
lga_summary = [{"Total Schools": total_schools, "Total Students": total_students,\
                "Total Budget": total_budget, "Average Maths Score": av_maths_score,\
                "Average Reading Score": av_read_score, "% Passing Maths": maths_percent_pass,\
                "% Passing Reading": read_percent_pass, "% Overall Pass": both_percent_pass}]

#create dataframe
lga_summary_df = pd.DataFrame(lga_summary)

#format required columns for decimal places and currency
lga_summary_df["Total Students"] = lga_summary_df["Total Students"].astype(int).map("{:,}".format)
lga_summary_df["Total Budget"] = lga_summary_df["Total Budget"].astype(float).map("${:,.2f}".format)

#view
lga_summary_df


,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Pass
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


In [9]:
#group merged data by school name
schools_grouped = schools_merged_df.groupby(["school_name"])

In [13]:
#calculations per school

#total students
school_students = pd.DataFrame(schools_grouped["Student ID"].nunique())
school_students = school_students.rename(columns={"Student ID": "Total Students"})
school_students

#total school budget
first_school = schools_merged_df.groupby("school_name").first()
school_budget = first_school["budget"]


#per student budget
school_size = first_school["size"]
per_student = pd.DataFrame(school_budget/school_size)
per_student = per_student.rename(columns={0: "Per Student Budget"})
#per_student = per_student.astype(int)
per_student

#average maths score
school_av_maths_score = pd.DataFrame(schools_grouped["maths_score"].mean())
school_av_maths_score = school_av_maths_score.rename(columns={"maths_score": "Average Maths Score"})
school_av_maths_score

#average reading score
school_av_read_score = pd.DataFrame(schools_grouped["reading_score"].mean())
school_av_read_score = school_av_read_score.rename(columns={"reading_score": "Average Reading Score"})
school_av_read_score

#percentage passing maths
math_group_pass_df = schools_merged_df.loc[(schools_merged_df["maths_score"] >= 50)]
maths_grouped_df = math_group_pass_df.groupby(["school_name"])
math_pass_sch = maths_grouped_df["maths_score"].count()
math_attempted = schools_grouped["maths_score"].count()
math_percent_final = (math_pass_sch / math_attempted)*100
math_percent_final

#percentage passing reading
read_group_pass_df = schools_merged_df.loc[(schools_merged_df["reading_score"] >= 50)]
read_grouped_df = read_group_pass_df.groupby(["school_name"])
read_pass_sch = read_grouped_df["reading_score"].count()
read_attempted = schools_grouped["reading_score"].count()
read_percent_final = (read_pass_sch / read_attempted)*100
read_percent_final

#percentage passing both maths and reading
all_group_pass_df = schools_merged_df.loc[(schools_merged_df["maths_score"] >= 50)\
                                          & (schools_merged_df["reading_score"] >= 50)]
all_grouped_df = all_group_pass_df.groupby(["school_name"])
all_pass_sch = all_grouped_df["Student ID"].count()
all_attempted = schools_grouped["Student ID"].count()
all_percent_final = (all_pass_sch / all_attempted)*100
all_percent_final



school_name
Bailey High School       80.084405
Cabrera High School      80.785791
Figueroa High School     67.650051
Ford High School         67.469880
Griffin High School      81.335150
Hernandez High School    66.364617
Holden High School       78.922717
Huang High School        66.712376
Johnson High School      67.191766
Pena High School         79.209979
Rodriguez High School    79.419855
Shelton High School      78.875639
Thomas High School       69.480122
Wilson High School       67.455103
Wright High School       79.722222
Name: Student ID, dtype: float64

In [14]:
school_type = schools_grouped["type"].first()
school_budget = schools_grouped["budget"].first()
school_results = [school_type, school_students, school_budget, per_student, school_av_maths_score,\
                  school_av_read_score, math_percent_final, read_percent_final, all_percent_final]

school_summary = pd.concat(school_results, axis=1)
school_summary = school_summary.rename(columns={"type": "School Type", "budget": "Total School Budget",\
                                                "maths_score": "% Passing Maths", "reading_score": "% Passing Reading",\
                                                "Student ID": "% Overall Passing"})
school_summary_int = pd.DataFrame(school_summary)
school_summary_df = pd.DataFrame(school_summary_int)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].astype(float).map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].astype(float).map("${:,.2f}".format)

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766


In [15]:
school_summary_int["% Passing Maths"].dtype

dtype('float64')

In [16]:
top_overall = school_summary.sort_values("% Overall Passing", ascending=False)
top_overall.head()


# To sort from highest to lowest, ascending=False must be passed in
#meals_taxes_df = taxes_df.sort_values("Meals", ascending=False)
#meals_taxes_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


In [17]:
top_overall = school_summary.sort_values("% Overall Passing", ascending=True)
top_overall.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


In [18]:
##year_maths_av = schools_merged_df.groupby(["school_name", "year"])["maths_score"].mean()
##year_maths_av

##year_maths_df = pd.DataFrame(year_maths_av)
##year_maths_df

#year_maths_av = pd.DataFrame(schools_merged_df.groupby(["school_name", "year"])["maths_score"].mean())
#year_maths_grouped = year_maths_av.groupby(["school_name"])
#year_maths_grouped.head()

maths_nine = schools_merged_df.loc[(schools_merged_df['year'] == 9)]
maths_nine_school = pd.DataFrame(maths_nine.groupby(["school_name"])["maths_score"].mean())
maths_nine_school = maths_nine_school.rename(columns={"maths_score": "Year 9"})
#maths_nine_school

maths_ten = schools_merged_df.loc[(schools_merged_df['year'] == 10)]
maths_ten_school = pd.DataFrame(maths_ten.groupby(["school_name"])["maths_score"].mean())
maths_ten_school = maths_ten_school.rename(columns={"maths_score": "Year 10"})
#maths_ten_school

maths_eleven = schools_merged_df.loc[(schools_merged_df['year'] == 11)]
maths_eleven_school = pd.DataFrame(maths_eleven.groupby(["school_name"])["maths_score"].mean())
maths_eleven_school = maths_eleven_school.rename(columns={"maths_score": "Year 11"})
#maths_eleven_school

maths_twelve = schools_merged_df.loc[(schools_merged_df['year'] == 12)]
maths_twelve_school = pd.DataFrame(maths_twelve.groupby(["school_name"])["maths_score"].mean())
maths_twelve_school = maths_twelve_school.rename(columns={"maths_score": "Year 12"})
#maths_twelve_school


years = [maths_nine_school, maths_ten_school, maths_eleven_school, maths_twelve_school]

years_summary = pd.concat(years,axis=1)
#years_summary = years_summary.rename(columns={years_summary.columns[1]: "Year 9", years_summary.columns[2]: "Year 10"})
years_summary
#chool_summary = pd.concat(school_results, axis=1)

#read_group_pass_df = schools_merged_df.loc[(schools_merged_df["reading_score"] >= 50)]
#read_grouped_df = read_group_pass_df.groupby(["school_name"])

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393


In [19]:
read_nine = schools_merged_df.loc[(schools_merged_df['year'] == 9)]
read_nine_school = pd.DataFrame(read_nine.groupby(["school_name"])["reading_score"].mean())
read_nine_school = read_nine_school.rename(columns={"reading_score": "Year 9"})
#read_nine_school

read_ten = schools_merged_df.loc[(schools_merged_df['year'] == 10)]
read_ten_school = pd.DataFrame(read_ten.groupby(["school_name"])["reading_score"].mean())
read_ten_school = read_ten_school.rename(columns={"reading_score": "Year 10"})
#read_ten_school

read_eleven = schools_merged_df.loc[(schools_merged_df['year'] == 11)]
read_eleven_school = pd.DataFrame(read_eleven.groupby(["school_name"])["reading_score"].mean())
read_eleven_school = read_eleven_school.rename(columns={"reading_score": "Year 11"})
#read_eleven_school

read_twelve = schools_merged_df.loc[(schools_merged_df['year'] == 12)]
read_twelve_school = pd.DataFrame(read_twelve.groupby(["school_name"])["reading_score"].mean())
read_twelve_school = read_twelve_school.rename(columns={"reading_score": "Year 12"})
#read_twelve_school


read_years = [read_nine_school, read_ten_school, read_eleven_school, read_twelve_school]

read_years_summary = pd.concat(read_years,axis=1)
read_years_summary


,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521


In [49]:
#spending_bins = [0,585,630,645,680]
#spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]
##spending_ranges = pd.DataFrame(pd.cut(per_student["Per Student Budget"], spending_bins, labels=spending_labels))
##spending_rangess
#schools_merged_df["Per Student"]= schools_merged_df["budget"]/schools_merged_df["size"]
#schools_merged_df["Spending Ranges (Per Student)"]= pd.cut(schools_merged_df["Per Student"], spending_bins, labels=spending_labels)
#schools_merged_df.head()

In [50]:
#spending_groups = spending_ranges.groupby("Per Student Budget")
#spending_groups.count()

#spending_results = [school_av_maths_score, school_av_read_score, math_percent_final, read_percent_final, all_percent_final]
#spending_summary = pd.concat(spending_results, axis=1)
#spending_summary

In [51]:
#schools_spending_grouped = schools_merged_df.groupby("Spending Ranges (Per Student)")
#schools_spending_grouped.head()

In [52]:
#spend_av_maths_score = pd.DataFrame(schools_spending_grouped["maths_score"].mean())
#spend_av_maths_score = spend_av_maths_score.rename(columns={"maths_score": "Average Maths Score"})
#spend_av_maths_score

In [53]:
spending_bins2 = [0,585,630,645,680]
spending_labels2 = ["<$585", "$585-630", "$630-645", "$645-680"]
#school_summary["Per Student Budget"] = school_summary["Per Student Budget"].str.replace("$", "")
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].astype(float)
school_summary["Spending Ranges (Per Student)"]= pd.cut(school_summary["Per Student Budget"],\
                                                           spending_bins2, labels=spending_labels2)
school_summary.head()
#spending_ranges2 = pd.cut(["Per Student Budget"], spending_bins2, labels=spending_labels2))
#spending_ranges2

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",628.0,72.352894,71.008842,91.639871,87.379421,80.084405,$585-630
Cabrera High School,Independent,1858,"$1,081,356.00",582.0,71.657158,71.359526,90.850377,89.074273,80.785791,<$585
Figueroa High School,Government,2949,"$1,884,411.00",639.0,68.698542,69.077993,81.654798,82.807731,67.650051,$630-645
Ford High School,Government,2739,"$1,763,916.00",644.0,69.091274,69.572472,82.438846,82.219788,67.469880,$630-645
Griffin High School,Independent,1468,"$917,500.00",625.0,71.788147,71.245232,91.212534,88.487738,81.335150,$585-630


In [54]:
spending_math_scores = pd.DataFrame(school_summary.groupby(["Spending Ranges (Per Student)"])\
                                    ["Average Maths Score"].mean())
spending_math_scores

spending_reading_scores = pd.DataFrame(school_summary.groupby(["Spending Ranges (Per Student)"])\
                                       ["Average Reading Score"].mean())
spending_reading_scores

spending_passing_math = pd.DataFrame(school_summary.groupby(["Spending Ranges (Per Student)"])\
                                     ["% Passing Maths"].mean())
spending_passing_math

spending_passing_reading = pd.DataFrame(school_summary.groupby(["Spending Ranges (Per Student)"])\
                                        ["% Passing Reading"].mean())
spending_passing_reading

overall_passing_spending = pd.DataFrame(school_summary.groupby(["Spending Ranges (Per Student)"])\
                                        ["% Overall Passing"].mean()) 

#school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].astype(float).map("${:,.2f}".
overall_passing_spending

,% Overall Passing
Spending Ranges (Per Student),
<$585,76.721458
$585-630,79.876293
$630-645,71.004977
$645-680,66.756253


In [55]:
spending_results = [spending_math_scores, spending_reading_scores, spending_passing_math, spending_passing_reading,\
                    overall_passing_spending]

spending_summary = pd.concat(spending_results, axis=1)
#spending_summary["Average Maths Score"] = spending_summary["Average Maths Score"].astype(float).map("{:.2f}")
spending_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.364587,70.716577,88.835926,86.390517,76.721458
$585-630,72.065868,71.031297,91.518824,87.292423,79.876293
$630-645,69.854807,69.838814,84.686139,83.763585,71.004977
$645-680,68.884391,69.045403,81.568470,81.769716,66.756253


In [48]:
spending_summary["% Passing Maths"].dtype

dtype('float64')

In [56]:
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#school_summary["Per Student Budget"] = school_summary["Per Student Budget"].astype(float)
school_summary["School Size"]= pd.cut(school_summary["Total Students"],\
                                                           size_bins, labels=size_labels)
school_summary.head()
#spending_ranges2 = pd.cut(["Per Student Budget"], spending_bins2, labels=spending_labels2))
#spending_ranges2

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
school_name,,,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",628.0,72.352894,71.008842,91.639871,87.379421,80.084405,$585-630,Large (2000-5000)
Cabrera High School,Independent,1858,"$1,081,356.00",582.0,71.657158,71.359526,90.850377,89.074273,80.785791,<$585,Medium (1000-2000)
Figueroa High School,Government,2949,"$1,884,411.00",639.0,68.698542,69.077993,81.654798,82.807731,67.650051,$630-645,Large (2000-5000)
Ford High School,Government,2739,"$1,763,916.00",644.0,69.091274,69.572472,82.438846,82.219788,67.469880,$630-645,Large (2000-5000)
Griffin High School,Independent,1468,"$917,500.00",625.0,71.788147,71.245232,91.212534,88.487738,81.335150,$585-630,Medium (1000-2000)


In [57]:
size_math_scores = pd.DataFrame(school_summary.groupby(["School Size"])\
                                    ["Average Maths Score"].mean())
size_math_scores

size_reading_scores = pd.DataFrame(school_summary.groupby(["School Size"])\
                                       ["Average Reading Score"].mean())
size_reading_scores

size_passing_math = pd.DataFrame(school_summary.groupby(["School Size"])\
                                     ["% Passing Maths"].mean())
size_passing_math

size_passing_reading = pd.DataFrame(school_summary.groupby(["School Size"])\
                                        ["% Passing Reading"].mean())
size_passing_reading

overall_passing_size = pd.DataFrame(school_summary.groupby(["School Size"])\
                                        ["% Overall Passing"].mean()) 

overall_passing_size

,% Overall Passing
School Size,
Small (<1000),79.066348
Medium (1000-2000),78.039785
Large (2000-5000),70.293507


In [58]:
size_results = [size_math_scores, size_reading_scores, size_passing_math, size_passing_reading,\
                    overall_passing_size]

size_summary = pd.concat(size_results, axis=1)
size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


In [59]:
type_math_scores = pd.DataFrame(school_summary.groupby(["School Type"])\
                                    ["Average Maths Score"].mean())
type_math_scores

type_reading_scores = pd.DataFrame(school_summary.groupby(["School Type"])\
                                       ["Average Reading Score"].mean())
type_reading_scores

type_passing_math = pd.DataFrame(school_summary.groupby(["School Type"])\
                                     ["% Passing Maths"].mean())
type_passing_math

type_passing_reading = pd.DataFrame(school_summary.groupby(["School Type"])\
                                        ["% Passing Reading"].mean())
type_passing_reading

overall_passing_type = pd.DataFrame(school_summary.groupby(["School Type"])\
                                        ["% Overall Passing"].mean()) 

overall_passing_type

,% Overall Passing
School Type,
Government,70.698993
Independent,76.973340


In [60]:
type_results = [type_math_scores, type_reading_scores, type_passing_math, type_passing_reading,\
                    overall_passing_type]

type_summary = pd.concat(type_results, axis=1)
type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
